In [1]:
from models.waffleiron.segmenter import Segmenter
import torch
from datasets import LIST_DATASETS, Collate
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from collections import OrderedDict
import warnings
import copy
import random
import numpy as np
import os

import argparse
import wandb
from torchmetrics.classification import MulticlassJaccardIndex
import torchmetrics
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

import torchhd
from torchhd.models import Centroid
from torchhd import embeddings

Using torch.scatter_reduce for 3D to 2D projection.
Using torch.scatter_reduce for 3D to 2D projection.


In [71]:
class Feature_Extractor:
    def __init__(self, input_channels=5, feat_channels=768, depth=48, 
                 grid_shape=[[256, 256], [256, 32], [256, 32]], nb_class=16, layer_norm=True, 
                 device=torch.device("cpu"), early_exit = 48, **kwargs):
        self.model = Segmenter(
            input_channels=input_channels,
            feat_channels=feat_channels,
            depth=depth,
            grid_shape=grid_shape,
            nb_class=nb_class, # class for prediction
            #drop_path_prob=config["waffleiron"]["drop_path"],
            layer_norm=layer_norm,
        )

        classif = torch.nn.Conv1d(
            feat_channels, nb_class, 1 # So it fits 16 = nb_class but classifier is not used
        )
        torch.nn.init.constant_(classif.bias, 0)
        torch.nn.init.constant_(classif.weight, 0)
        self.model.classif = torch.nn.Sequential(
            torch.nn.BatchNorm1d(feat_channels),
            classif,
        )

        for p in self.model.parameters():
            p.requires_grad = False
        for p in self.model.classif.parameters():
            p.requires_grad = True

        def get_optimizer(parameters):
            return torch.optim.AdamW(
                parameters,
                lr=0.001,
                weight_decay=0.003,
            )

        optim = get_optimizer(self.model.parameters())
        self.device = device
        self.device_string = "cuda:0"
        self.num_classes = nb_class
        self.early_exit = early_exit
        self.kwargs = kwargs
    
    def load_pretrained(self, path):
        # Load pretrained model
        path_to_ckpt = path
        checkpoint = torch.load(path_to_ckpt,
            map_location=self.device_string)
        state_dict = checkpoint["net"]  # Adjust key as needed
        new_state_dict = OrderedDict()

        for k, v in state_dict.items():
            new_key = k.replace("module.", "")  # Remove "module." prefix
            new_state_dict[new_key] = v

        self.model.load_state_dict(new_state_dict)

        print(
            f"Checkpoint loaded on {self.device_string}: {path_to_ckpt}"
        )

        if self.device_string != 'cpu':
            torch.cuda.set_device(self.device_string) # cuda:0
            self.model = self.model.cuda(self.device_string) # cuda:0

        self.model.eval()

    def forward_model(self, it, batch):
        pts = batch["points"]
        feat = batch["feat"]
        labels = batch["labels_orig"]
        cell_ind = batch["cell_ind"]
        occupied_cell = batch["occupied_cells"]
        neighbors_emb = batch["neighbors_emb"]
        if self.device_string != 'cpu':
            feat = feat.cuda(0, non_blocking=True)
            labels = labels.cuda(0, non_blocking=True)
            batch["upsample"] = [
                up.cuda(0, non_blocking=True) for up in batch["upsample"]
            ]
            cell_ind = cell_ind.cuda(0, non_blocking=True)
            occupied_cell = occupied_cell.cuda(0, non_blocking=True)
            neighbors_emb = neighbors_emb.cuda(0, non_blocking=True)
        net_inputs = (feat, cell_ind, occupied_cell, neighbors_emb)

        if self.device_string != 'cpu':
            with torch.autocast("cuda", enabled=True):
                # Logits
                with torch.no_grad():
                    out = self.model(*net_inputs, self.early_exit)
                    encode, tokens, out = out[0], out[1], out[2]
                    pred_label = out.max(1)[1]

                    # Only return samples that are not noise
                    #torch.cuda.synchronize(device=self.device)
                    where = labels != 255
                    #torch.cuda.synchronize(device=self.device)
        else:
            with torch.no_grad():
                out = self.model(*net_inputs, self.early_exit)
                encode, tokens, out = out[0], out[1], out[2]
                pred_label = out.max(1)[1]

                # Only return samples that are not noise
                where = labels != 255

        pts = torch.tensor(pts[0]).to(self.device)
        where = where.cpu()
        return pts[where], tokens[0,:,where], labels[where], pred_label[0, where]

    def test(self, loader, total_voxels):        
        # Metric
        miou = MulticlassJaccardIndex(num_classes=self.num_classes, average=None).to(self.device, non_blocking=True)
        final_labels = torch.empty((total_voxels), device=self.device)
        final_pred = torch.empty((total_voxels), device=self.device)
        final_labels_sep = torch.empty((1, 90000), device=self.device)
        final_feat_sep = torch.empty((1, 768, 90000), device=self.device)
        final_soa_result_sep = torch.empty((1, 90000), device=self.device)
        final_points_sep = torch.empty((1, 3, 90000), device=self.device)
        num_voxels = []
        
        start_idx = 0
        for it, batch in tqdm(enumerate(loader), desc="SoA testing"):
            pts, features, labels, soa_result = self.forward_model(it, batch)
            shape_sample = labels.shape[0]
            num_voxels.append(shape_sample)
            labels_tensor = torch.reshape(torch.Tensor(labels), (1,shape_sample)).to(self.device)
            feat_tensor = torch.reshape(torch.Tensor(features), (1,768,shape_sample)).to(self.device)
            soa_tensor = torch.reshape(torch.Tensor(soa_result), (1,shape_sample)).to(self.device)
            pts_tensor = torch.reshape(torch.Tensor(pts), (1,3,shape_sample)).to(self.device)
            final_labels_sep = torch.concat((final_labels_sep, F.pad(input=labels_tensor, pad=(0, 90000 - shape_sample), mode='constant', value=0)))
            final_soa_result_sep = torch.concat((final_soa_result_sep, F.pad(input=soa_tensor, pad=(0, 90000 - shape_sample), mode='constant', value=0)))
            final_feat_sep = torch.concat((final_feat_sep, F.pad(input=feat_tensor, pad=(0, 90000 - shape_sample), mode='constant', value=0)))
            final_points_sep = torch.concat((final_points_sep, F.pad(input=pts_tensor, pad=(0, 90000 - shape_sample), mode='constant', value=0)))
            print(final_labels_sep.shape)
            print(final_soa_result_sep.shape)
            print(final_feat_sep.shape)
            print(final_points_sep.shape)

            #labels = labels.to(dtype = torch.int64, device = self.device, non_blocking=True)
            #soa_result = soa_result.to(device=self.device, non_blocking=True)
            #final_labels[start_idx:start_idx+shape_sample] = labels

            #final_pred[start_idx:start_idx+shape_sample] = soa_result

            #start_idx += shape_sample

            if it == 50:
                break

        #final_labels = final_labels[:start_idx]
        #final_pred = final_pred[:start_idx]

        #print("================================")

        #print('Pred FE', final_pred, "\tShape: ", final_pred.shape)
        #print('Label', final_labels, "\tShape: ", final_labels.shape)
        #accuracy = miou(final_pred, final_labels)
        #avg_acc = torch.mean(accuracy)
        #print(f'accuracy: {accuracy}')
        #print(f'avg acc: {avg_acc}')

        #cm = confusion_matrix(pred_hd, first_label, labels=torch.Tensor(range(0,15)))
        #print("Confusion matrix \n")
        #print(cm)

        #print("================================")

        return final_points_sep, final_labels_sep, final_soa_result_sep, final_feat_sep, num_voxels

In [72]:
fe = Feature_Extractor(nb_class=19)
fe.load_pretrained('/root/main/ScaLR/saved_models/ckpt_last_kitti.pth')

/tmp/ipykernel_29969/1350821209.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path_to_ckpt,


Checkpoint loaded on cuda:0: /root/main/ScaLR/saved_models/ckpt_last_kitti.pth


In [73]:
kwargs = {
    "rootdir": '/root/main/dataset/semantickitti',
    "input_feat": ["intensity", "xyz", "radius"],
    "voxel_size": 0.1,
    "num_neighbors": 16,
    "dim_proj": [2, 1, 0],
    "grids_shape": [[256, 256], [256, 32], [256, 32]],
    "fov_xyz": [[-64, -64, -8], [64, 64, 8]], # Check here
}

# Get datatset
DATASET = LIST_DATASETS.get('semantic_kitti')

dataset_train = DATASET(
    phase="train",
    **kwargs,
)

# Validation dataset
dataset_val = DATASET(
    phase="val",
    **kwargs,
)

num_classes = 19

path_pretrained = '/root/main/ScaLR/saved_models/ckpt_last_kitti.pth'

Using original split
Using original split


In [74]:
device = torch.device("cuda")
print("Using {} device".format(device))

train_loader = torch.utils.data.DataLoader(
        dataset_train,
        batch_size=1,
        pin_memory=True,
        drop_last=True,
        collate_fn=Collate(device=device),
        persistent_workers=False,
    )

val_loader = torch.utils.data.DataLoader(
    dataset_val,
    batch_size=1,
    pin_memory=True,
    drop_last=True,
    collate_fn=Collate(device=device),
    persistent_workers=False,
)

Using cuda device


In [75]:
pts, labels_train, soa_train, feat_train, vox_train = fe.test(train_loader, 50) # With the nuscenes hyperparameters...

SoA testing: 1it [00:03,  3.70s/it]

torch.Size([2, 90000])
torch.Size([2, 90000])
torch.Size([2, 768, 90000])
torch.Size([2, 3, 90000])


SoA testing: 2it [00:07,  3.92s/it]

torch.Size([3, 90000])
torch.Size([3, 90000])
torch.Size([3, 768, 90000])
torch.Size([3, 3, 90000])


SoA testing: 3it [00:11,  3.88s/it]

...

torch.Size([50, 90000])
torch.Size([50, 90000])
torch.Size([50, 768, 90000])
torch.Size([50, 3, 90000])


SoA testing: 50it [06:30, 11.07s/it]

torch.Size([51, 90000])
torch.Size([51, 90000])
torch.Size([51, 768, 90000])
torch.Size([51, 3, 90000])


SoA testing: 50it [06:43,  8.08s/it]

torch.Size([52, 90000])
torch.Size([52, 90000])
torch.Size([52, 768, 90000])
torch.Size([52, 3, 90000])


In [77]:
torch.tensor(vox_train)

tensor([46948, 50477, 55087, 47337, 33538, 53880, 62154, 58821, 56910, 59075,
        61729, 55668, 47670, 62586, 51217, 70168, 46743, 73190, 50894, 55858,
        47573, 72963, 38323, 53441, 47561, 52344, 46960, 51288, 54708, 53011,
        52989, 50063, 54923, 46465, 52676, 50247, 54651, 41890, 55716, 68641,
        44248, 51205, 55295, 50700, 41873, 56819, 69015, 38397, 44931, 35259,
        63025])

In [80]:
torch.save(labels_train[1:], 'labels_train_semkitti.pt')
torch.save(soa_train[1:], 'soa_train_semkitti.pt')
torch.save(feat_train[1:], 'feat_train_semkitti.pt')
torch.save(pts[1:], 'pts_train_semkitti.pt')

In [81]:
torch.save(torch.tensor(vox_train), 'voxels_train_semkitti.pt')

In [82]:
pts, labels, soa, feat, vox = fe.test(val_loader, 50) # With the nuscenes hyperparameters...

SoA testing: 1it [00:03,  3.39s/it]

torch.Size([2, 90000])
torch.Size([2, 90000])
torch.Size([2, 768, 90000])
torch.Size([2, 3, 90000])


SoA testing: 2it [00:07,  3.85s/it]

torch.Size([3, 90000])
torch.Size([3, 90000])
torch.Size([3, 768, 90000])
torch.Size([3, 3, 90000])


SoA testing: 3it [00:11,  3.88s/it]

...

torch.Size([50, 90000])
torch.Size([50, 90000])
torch.Size([50, 768, 90000])
torch.Size([50, 3, 90000])


SoA testing: 50it [06:47, 12.42s/it]

torch.Size([51, 90000])
torch.Size([51, 90000])
torch.Size([51, 768, 90000])
torch.Size([51, 3, 90000])


SoA testing: 50it [06:58,  8.37s/it]

torch.Size([52, 90000])
torch.Size([52, 90000])
torch.Size([52, 768, 90000])
torch.Size([52, 3, 90000])


In [224]:
torch.save(labels[1:], 'labels_test_semkitti.pt')
torch.save(soa[1:], 'soa_test_semkitti.pt')
torch.save(feat[1:], 'feat_test_semkitti.pt')
torch.save(torch.tensor(vox), 'voxels_test_semkitti.pt')
torch.save(pts[1:], 'pts_test_semkitti.pt')

# Check if the two models have the same weights?

In [71]:
# SemanticKitti
fe.model

Segmenter(
  (embed): Embedding(
    (norm): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1): Conv1d(5, 768, kernel_size=(1,), stride=(1,))
    (conv2): Sequential(
      (0): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv2d(5, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
      (4): Conv2d(768, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (final): Conv1d(1536, 768, kernel_size=(1,), stride=(1,))
  )
  (waffleiron): WaffleIron(
    (channel_mix): ModuleList(
      (0-47): 48 x ChannelMix(
        (norm): myLayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Conv1d(768, 768, kernel_size=(1,), stride=(1,))
          (1): ReLU(inplace=True)
          (2): Conv1d(768, 768, kernel_size=(1,), stride=(1,))
        )
 

In [53]:
fe_2 = Feature_Extractor(nb_class=16)
fe_2.load_pretrained('/root/main/ScaLR/saved_models/ckpt_last_scalr.pth')

/tmp/ipykernel_15601/2784029278.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path_to_ckpt,


Checkpoint loaded on cuda:0: /root/main/ScaLR/saved_models/ckpt_last_scalr.pth


In [72]:
import torch
from torch.nn import Module

# Assuming `model1` and `model2` are your two models
def compare_model_weights(model1: Module, model2: Module):
    for (name1, param1), (name2, param2) in zip(model1.named_parameters(), model2.named_parameters()):
        if name1 != name2:
            print(f"Parameter names differ: {name1} vs {name2}")
            return False
        if not torch.equal(param1.data, param2.data):
            print(f"Weights differ in {name1}")
            return False
    print("All weights are didentical!")
    return True

# Call the function with your two models
result = compare_model_weights(fe.model, fe_2.model)

Weights differ in classif.0.weight


In [96]:
x = torch.ones((5,))
F.pad(input=x, pad=(0, 5), mode='constant', value=0)

tensor([1., 1., 1., 1., 1., 0., 0., 0., 0., 0.])

# See nuscenes number of voxels

In [2]:
import numpy as np
voxels = np.load('/root/main/ScaLR/debug/nuscenes/num_voxels.npy')

In [3]:
voxels

array([18197., 16838., 18008., 17197., 17114., 16733., 17014., 16558.,
       16587., 14200., 17857.], dtype=float32)